# Inicializar

In [8]:
import gurobipy as gp
from gurobipy import GRB
import os
import re
import pandas as pd

In [9]:
# Mostrar el modelo
def displayModel(model,name):
    try:
        os.remove(name)
    except OSError:
        pass
    model.write(name) # Escribir el modelo a un archivo .lp
    model.display()

In [10]:
L = 10
I = range(3) # 0: Low, 1: Medium, 2: High
c_values = {0: 5, 1: 12, 2: 20}

In [11]:
def mostrarResultadosTabla(cantidadNodos, costo, variablesDecision):

	if costo is None:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("No se encontró solución")
	else:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("Resultado de la Optimización:")
		print("====================================================")
		print("Costo Total: ", costo)
		print("Nodos activos: \n")

		xVars = dict()
		yVars = dict()
		zVars = dict()
		for var in variablesDecision.keys():
			if var.startswith("x"):
				xVars[var] = variablesDecision[var]
			if var.startswith("y"):
				yVars[var] = variablesDecision[var]
			if var.startswith("z"):
				zVars[var] = variablesDecision[var]

		xactiveNodes = []
		yactiveNodes = []
		zactiveNodes = []
		for u in range(cantidadNodos):
			valoresx = []
			valoresy = []
			valoresz = []
			patternx = re.compile(r'^x\[' + str(u) + r',')
			patterny = re.compile(r'^y\[' + str(u) + r',')
			patternz = re.compile(r'^z\[' + str(u) + r',')
			for var in xVars.keys():
				if patternx.match(var):
					valoresx.append(int(xVars[var]))
			for var in yVars.keys():
				if patterny.match(var):
					valoresy.append(int(yVars[var]))
			for var in zVars.keys():
				if patternz.match(var):
					valoresz.append(int(zVars[var]))
			xactiveNodes.append(valoresx)
			yactiveNodes.append(valoresy)
			zactiveNodes.append(valoresz)

		columns_titles_x = ["Low Cost", "Mid Cost", "High Cost"]
		columns_titles_y = ["Series", "Paralelo"]
		row_index = [u+1 for u in range(cantidadNodos)]

		tablax = pd.DataFrame(
			xactiveNodes, columns=columns_titles_x, index=row_index)
		tablay = pd.DataFrame(
			yactiveNodes, columns=columns_titles_y, index=row_index)
		tablaz = pd.DataFrame(
			zactiveNodes, index=row_index)
		print(tablax)
		print(tablay)
		print(tablaz)
		print("====================================================")

# Modelo base

In [12]:
def base_model(N):

	if N < 4:
		raise ValueError("El número de nodos debe ser al menos 4.")

	model = gp.Model(f"General_Model_{N}_Nodos")

	# Parámetros
	U = range(N) # Nodos a desplegar

	#####################################################################

	# Variables de decisión
	x = model.addVars(U, I, vtype=GRB.BINARY, name="x") # Variable de decisión para el tipo de nodos

	# Variables auxiliares
	nodesCost = model.addVar(vtype=GRB.CONTINUOUS, name="nodesCost") # Costo de los nodos desplegados
	model.addConstr(nodesCost == gp.quicksum(c_values[i] * x[u, i] for i in I for u in U), name="NodesCost_def")

	linksCost = model.addVar(vtype=GRB.CONTINUOUS, name="linksCost")
	model.addConstr(linksCost == 0, name="LinksCost_General") # Costo de los enlaces para el modelo general 0

	#####################################################################

	# Restricción de unicidad de nodos: cada nodo puede ser de un solo tipo
	model.addConstrs(
		(gp.quicksum(x[u, i] for i in I) == 1 for u in U), name="Unicidad_i"
	)

	#####################################################################

	model.setParam('OutputFlag', 0)  # No mostrar salida por defecto de Gurobi
	model.setObjective(nodesCost+linksCost, GRB.MINIMIZE)

	model.optimize()

	return model

# Modelos heredados

In [13]:
def serie_model(model_base, N):

	U = range(N)

	# Copiar el modelo base
	model = model_base.copy()

	############################################################

	# Recuperar variables
	x = { tuple(map(int, var.varName.split('[')[1].split(']')[0].split(','))): var for var in model.getVars() if "x" in var.varName }

	############################################################

	N_s = model.addVar(vtype=GRB.INTEGER, name="N_s") # Número de nodos en serie
	model.addConstr(N_s == gp.quicksum(x[u, i] for u in U for i in I), name="Ns_def") # Número de nodos en serie

	############################################################

	# Actualizar restricciones de costos de enlaces
	linksCost_Condition = model.getConstrByName("LinksCost_General")
	if linksCost_Condition is not None:
		model.remove(linksCost_Condition)

	linksCost = model.getVarByName("linksCost")
	if linksCost is None:
		raise ValueError("No se encontró la variable de costo de enlaces.")

	model.addConstr(linksCost == L*(N_s-1), name="LinksCost_Serie") # Costo de los enlaces en serie

	# Optimizar el modelo
	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = {var.varName: var.x for var in model.getVars()}
		return model.objVal, variablesDecision
	else:
		raise Exception("No se encontró una solución óptima.")

In [15]:
N = 5 # Número de nodos a desplegar

modeloGeneral = base_model(N)
costoSerie, variablesSerie = serie_model(modeloGeneral, N)
print("====================================================")
print("Modelo en Serie")
print("Costo Total: ", costoSerie)
print("====================================================")
print("Resultados de la Optimización:")
for var in variablesSerie.keys():
    print(var, ":", variablesSerie[var])

Modelo en Serie
Costo Total:  65.0
Resultados de la Optimización:
x[0,0] : 1.0
x[0,1] : 0.0
x[0,2] : 0.0
x[1,0] : 1.0
x[1,1] : 0.0
x[1,2] : 0.0
x[2,0] : 1.0
x[2,1] : 0.0
x[2,2] : 0.0
x[3,0] : 1.0
x[3,1] : 0.0
x[3,2] : 0.0
x[4,0] : 1.0
x[4,1] : 0.0
x[4,2] : 0.0
nodesCost : 25.0
linksCost : 40.0
N_s : 5.0


In [7]:
def parallel_model(model_base, N):
	"""
	Convierte un modelo base híbrido en un modelo solo paralelo.

	Args:
		model_base: Modelo base híbrido (Gurobi Model).
		N: Número de nodos (debe ser un entero mayor o igual a 3).

	Returns:
		objVal: Valor óptimo de la función objetivo.
		variablesDecision: Diccionario con los valores de las variables de decisión.

	Raises:
		ValueError: Si N es menor a 3 o no es un entero.
		Exception: Si no se encuentra una solución óptima.
	"""
	if not isinstance(N, int) or N < 3:
		raise ValueError("El modelo solo paralelo requiere al menos 3 nodos.")

	U = range(N)

	# Copiar el modelo base
	model = model_base.copy()

	# Agregar restricciones específicas del modelo paralelo
	N_s = model.getVarByName("N_s")  # Obtener variable N_s del modelo base
	if N_s is None:
		raise ValueError("La variable N_s no se encontró en el modelo base.")
	model.addConstr(N_s == 0, name="Solo_Paralelo")  # Número de nodos en serie igual a 0

	# Asegurar que todos los nodos pertenezcan a la subred 1
	z = {tuple(map(int, var.varName.split('[')[1].split(']')[0].split(','))): var
		 for var in model.getVars() if "z" in var.varName}
	model.addConstrs(
		(z[u, 1] == 1 for u in U), name="Una_Subred"
	)

	# Optimizar el modelo
	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = {var.varName: var.x for var in model.getVars()}
		return model.objVal, variablesDecision
	else:
		raise Exception("No se encontró una solución óptima.")

In [8]:
def hibrid_model(model_base, N, L):

	if N < 3:
		raise ValueError("El número de nodos debe ser al menos 3.")

	U = range(N) # Nodos a desplegar
	W = range(2) # 0: Serie, 1: Paralelo
	Subredes = N // 3 # Número de subredes a desplegar
	J = range(Subredes + 1) # Subredes a desplegar

	#################################################################

	# Copiar el modelo base
	model = model_base.copy() # Copiar el modelo base

	#############################################################

	y = model.addVars(U, W, vtype=GRB.BINARY, name="y") # Variable de decisión para el tipo de enlaces
	z = model.addVars(U, J, vtype=GRB.BINARY, name="z") # Variable de decisión para el tipo de subredes

	#####################################################################

	# Variables auxiliares
	N_s = model.addVar(vtype=GRB.INTEGER, name="N_s") # Número de nodos en serie
	N_p = model.addVar(vtype=GRB.INTEGER, name="N_p") # Número de nodos en paralelo
	k = model.addVar(vtype=GRB.INTEGER, name="k") # Variable auxiliar
	delta_Np = model.addVar(vtype=GRB.BINARY, name="delta_Np") # Variable binaria delta de kronecker
	alpha = model.addVars(J, vtype=GRB.BINARY, name="alpha") # Variables auxiliares para indicar si la subred j (j >= 1) está activa

	model.addConstr(N_s == gp.quicksum(y[u, 0] for u in U), name="Ns_def") # Número de nodos en serie
	model.addConstr(N_p == gp.quicksum(y[u, 1] for u in U), name="Np_def") # Número de nodos en paralelo
	model.addConstr(k * 2 == N_p * (N_p - 1), name="Quadratic_Exact") # Restricción cuadrática exacta
	model.addGenConstrIndicator(delta_Np, True, N_p == 0, name="Kronecker_1") # Restricción de Kronecker si Np == 0 -> delta_Np = 1
	model.addGenConstrIndicator(delta_Np, False, N_p >= 1, name="Kronecker_0") # Restricción de Kronecker si Np >= 1 -> delta_Np = 0
	model.addConstrs((alpha[j] >= z[u, j] for u in U for j in J if j > 0), name="Activar_Subred") # Activar subredes

	####################################################################

	# Restricción de unicidad de enlaces: cada nodo puede ser de un solo tipo
	model.addConstrs(
		(gp.quicksum(y[u, w] for w in W) == 1 for u in U), name="Unicidad_w"
	)

	# Restricción de unicidad de subredes: cada nodo puede ser de un solo tipo
	model.addConstrs(
		(gp.quicksum(z[u, j] for j in J) == 1 for u in U), name="Unicidad_j"
	)

	# 🔴 Esta restricción solo aplica en el modelo híbrido
	model.addConstr(N_p >= 3, name="Np_Condition") # Número de nodos en paralelo mínimo

	# Cada nodo serie pertenece a la subred 0
	model.addConstrs(
		(y[u, 0] == z[u, 0] for u in U),
		name="Serie_en_Subred_0"
	)

	# Restricción: Si la subred j >= 1 existe, debe tener al menos 3 nodos
	model.addConstrs(
		(gp.quicksum(z[u, j] for u in U) >= 3 * alpha[j] for j in J if j > 0),
		name="Subredes_Min_3"
	)

	#####################################################################

	# Costos de la red
	LinksCost = L * (N_s + k - delta_Np) # Costo de los enlaces desplegados
	model.setObjective(nodesCost, GRB.MINIMIZE)

	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = dict()
		for var in model.getVars():
			variablesDecision[var.varName] = var.x
		return model.objVal, variablesDecision, model
	else:
		raise Exception("No se encontró una solución óptima.")

# Pruebas

In [9]:
N = 4

## Serie

In [10]:
try:
	modeloBase = base_model(N)
	costoSerie, variablesDecisionSerie = serie_model(modeloBase, N)
	print("=====================================================")
	print("\nModelo solo serie")
	print("Costo: ", costoSerie)
	print("=====================================================")
	print("\nResultados:")
	for var in variablesDecisionSerie.keys():
		print(var, ":", variablesDecisionSerie[var])
except Exception as e:
	print(f"Error: {e}")

<gurobi.Model Continuous instance General_Model_4_Nodos_copy: 0 constrs, 0 vars, Parameter changes: OutputFlag=0>
Error: cannot unpack non-iterable NoneType object


## Paralelo

In [11]:
try:
	costoParalelo, variablesDecisionParalelo = parallel_model(modeloHibrido, N)
	print("\nModelo solo paralelo")
	mostrarResultadosTabla(N, costoParalelo, variablesDecisionParalelo)
except Exception as e:
	print(f"Error: {e}")

Error: name 'modeloHibrido' is not defined


## Hibrido

In [12]:
try:
	costoHibrido, variablesDecisionHibrido, modeloHibrido = base_model(N)
	print("Modelo híbrido")
	mostrarResultadosTabla(N, costoHibrido, variablesDecisionHibrido)
except Exception as e:
	print(f"Error: {e}")

Error: cannot unpack non-iterable Model object
